In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [118]:
#Takes list of path names as input, concats files and outputs two dataframes (echos and sensors)
def load_files(echo_files, sensor_files):
    echos = pd.DataFrame()
    sensors = pd.DataFrame()
    for path in echo_files:
        dfe = pd.read_csv(path)
        echos = pd.concat([echos, dfe])
        
    for path in sensor_files:
        dfs = pd.read_csv(path).reset_index()   
        sensors = pd.concat([sensors, dfs], ignore_index=True)
    
    sensors = sensors.drop(columns=['index'])
    return echos, sensors

def sensor_for_log_match(df_sensors):
    df_sensors.dataid = pd.to_datetime(df_sensors.dataid, format='%Y-%m-%d %H:%M:%S')
    df_sensors = df_sensors.groupby(pd.Grouper(key="dataid", freq="30T")).median()
    df_sensors = df_sensors.reset_index()
    df_sensors['date'] = df_sensors['dataid'].dt.date
    df_sensors.to_csv('sensors_log_match.csv', index = False)
    return df_sensors

In [119]:
echo, sensor = load_files(['leg2_echo_prepped.csv', 'leg3_echo_prepped.csv','leg4_echo_prepped.csv'],
           ['leg2_prepped.csv', 'leg3_prepped.csv','leg4_prepped.csv'])
sensor

,dataid,PCO2.H2O_mmm,PCO2.CO2_umm,NMEA.Humidity,NMEA.Trykk,FerryBox.SBE45_Salinity,FerryBox.Optode_Saturation,FerryBox.C3_Turbidity,FerryBox.C3_CHLAFluorescence,FerryBox.C3_Temperature,FerryBox.C3_CDOMFluorescence
0,2021-10-05 20:48:00,3.12,460.109985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-10-05 20:00:00,3.13,458.049988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-10-05 20:01:00,3.11,458.089996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-10-05 20:03:00,3.12,458.230011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-10-05 20:04:00,3.13,458.299988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
131286,2022-02-24 00:01:00,NaN,NaN,NaN,NaN,1.651000,74.379997,136.979996,-0.27,43.590000,-0.14
131287,2022-02-25 09:57:00,NaN,NaN,NaN,NaN,1.633000,67.050003,106.300003,-0.22,41.259998,-0.04
131288,2022-02-25 15:08:00,NaN,NaN,NaN,NaN,1.635000,66.089996,179.179993,-0.31,41.200001,-0.01
131289,2022-02-27 22:06:00,NaN,NaN,NaN,NaN,35.590000,95.480003,-0.380000,-0.45,27.709999,-0.20


In [120]:
def combine_files(df_echo, df_sensors):
    # datetime type is converted as object when saved to csv, convert back to datetime after reading
    df_echo.dataid = pd.to_datetime(df_echo.dataid, format='%Y-%m-%d %H:%M:%S')
    df_sensors.dataid = pd.to_datetime(df_sensors.dataid, format='%Y-%m-%d %H:%M:%S')
    
    # fix outliers due to freshwater rinsing of the ferrybox system every night
    replace_freshwater(df_sensors)
    
    # group in intervals of 30 min
    df_sensors = df_sensors.groupby(pd.Grouper(key="dataid", freq="30T")).median()
    df_sensors = df_sensors.reset_index()

    # join sensor and echo dataframes
    final_df = df_echo.merge(df_sensors, on='dataid', how='outer')
    
   
    # Split datetime to date and time separately
    # Date to integer YYYY-MM-DD
    final_df['date'] = final_df['dataid'].dt.date
    final_df['day']=final_df['dataid'].dt.date.apply(lambda x: int(x.strftime("%d")))
    final_df['month']=final_df['dataid'].dt.date.apply(lambda x: int(x.strftime("%m")))
    

    # Time to float, e.g 07:30 == 7.5 and 21:00 == 21
    final_df['time'] = final_df['dataid'].dt.time
    final_df['time']=final_df['time'].apply(lambda x: x.hour + x.minute/60)
    final_df['time']=final_df['time'].astype('float')

    # Drop dataid
    final_df = final_df.drop(columns=['dataid'])
    
    # Remove rows with NaN values in target (TOTAL)
    final_df = final_df.dropna(subset = ['TOTAL'])
    final_df = final_df.fillna(final_df.median())

    return final_df


In [121]:
def replace_freshwater(df):
    # Get indices where salinity < 30
    result = np.where(df['FerryBox.SBE45_Salinity']<=30)
    idxs = result[0]
    # Ferrybox sensors that would be affected by the rinse
    sensors = ['FerryBox.C3_CDOMFluorescence', 'FerryBox.C3_Temperature', 'FerryBox.C3_CHLAFluorescence',
               'FerryBox.C3_Turbidity','FerryBox.Optode_Saturation', 'FerryBox.SBE45_Salinity']
    
    # Replace with median for column
    for s in sensors:   
        df.loc[idxs, s] = df[s].median()
    
    

In [122]:
final_df = combine_files(echo, sensor)

In [123]:
final_df

,LATITUDE,LONGITUD,PDMEAN,TOTAL,sf_depth,PCO2.H2O_mmm,PCO2.CO2_umm,NMEA.Humidity,NMEA.Trykk,FerryBox.SBE45_Salinity,FerryBox.Optode_Saturation,FerryBox.C3_Turbidity,FerryBox.C3_CHLAFluorescence,FerryBox.C3_Temperature,FerryBox.C3_CDOMFluorescence,date,day,month,time
0,27.20153,-16.88955,7.5,0.0000,-3611.0,0.42,425.709991,75.783337,1011.825012,35.825001,95.339996,0.91,-0.505,28.580,-0.16,2021-10-05,5,10,10.0
1,27.20153,-16.88955,15.0,4.6863,-3611.0,0.42,425.709991,75.783337,1011.825012,35.825001,95.339996,0.91,-0.505,28.580,-0.16,2021-10-05,5,10,10.0
2,27.20153,-16.88955,25.0,24.4440,-3611.0,0.42,425.709991,75.783337,1011.825012,35.825001,95.339996,0.91,-0.505,28.580,-0.16,2021-10-05,5,10,10.0
3,27.20153,-16.88955,35.0,30.9997,-3611.0,0.42,425.709991,75.783337,1011.825012,35.825001,95.339996,0.91,-0.505,28.580,-0.16,2021-10-05,5,10,10.0
4,27.20153,-16.88955,45.0,33.6487,-3611.0,0.42,425.709991,75.783337,1011.825012,35.825001,95.339996,0.91,-0.505,28.580,-0.16,2021-10-05,5,10,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284567,-23.19454,-40.66192,685.0,35.0402,-1894.0,0.42,425.709991,85.858334,1008.785004,35.882000,94.059998,-0.38,-0.450,28.405,-0.15,2022-02-19,19,2,21.0
284568,-23.19454,-40.66192,695.0,45.1827,-1894.0,0.42,425.709991,85.858334,1008.785004,35.882000,94.059998,-0.38,-0.450,28.405,-0.15,2022-02-19,19,2,21.0
284569,-23.19454,-40.66192,705.0,351.3631,-1894.0,0.42,425.709991,85.858334,1008.785004,35.882000,94.059998,-0.38,-0.450,28.405,-0.15,2022-02-19,19,2,21.0
284570,-23.19454,-40.66192,715.0,42.6268,-1894.0,0.42,425.709991,85.858334,1008.785004,35.882000,94.059998,-0.38,-0.450,28.405,-0.15,2022-02-19,19,2,21.0


In [124]:
data = {'Port':['La Coruna', 'Lisbon', 'Cadiz', 'Las Palmas', 'Willemstad',
                'Port Royal', 'Havana', 'Nassau', 'Miami', 'New York',
                'New Port', 'Horta', 'Rio'], 'Date from':['28.08.2021', '02.09.2021', '09.09.2021',
                '30.09.2021', '04.11.2021', '13.11.2021', '24.11.2021', '02.12.2021', '07.12.2021',
                '18.12.2021', '05.01.2022', '22.01.2022', '23.02.2022'],
                'Date to':['29.08.2021', '05.09.2021', '16.09.2021', '04.10.2021', '08.11.2021','17.11.2021',
                '28.11.2021', '05.12.2021', '10.12.2021', '04.01.2022', '08.01.2022', '24.01.2022', '26.02.2022']}

df_ports = pd.DataFrame(data)

df_ports['Date from'] = pd.to_datetime(df_ports['Date from'], dayfirst=True, format='%d.%m.%Y')
df_ports['Date to'] = pd.to_datetime(df_ports['Date to'], dayfirst=True, format='%d.%m.%Y')

def remove_ports_df(ports, df):
    df['date'] = pd.to_datetime(df['date'])
    for idx, row in ports.iterrows():
        from_date = row['Date from']
        to_date = row['Date to']
        #df = df.drop([(df['date'] <= from_date) & (df['date'] >= to_date)])
        df = df[(df['date'] < from_date) | (df['date'] > to_date)]
        
    #Fill median, drop rows with missing EK data
    df = df.dropna(subset=['TOTAL'])
    df = df.drop(columns=['date'])
    df = df.fillna(df.median())
    return df
    

In [125]:
final_df = remove_ports_df(df_ports, final_df)

In [126]:
final_df

,LATITUDE,LONGITUD,PDMEAN,TOTAL,sf_depth,PCO2.H2O_mmm,PCO2.CO2_umm,NMEA.Humidity,NMEA.Trykk,FerryBox.SBE45_Salinity,FerryBox.Optode_Saturation,FerryBox.C3_Turbidity,FerryBox.C3_CHLAFluorescence,FerryBox.C3_Temperature,FerryBox.C3_CDOMFluorescence,day,month,time
0,27.20153,-16.88955,7.5,0.0000,-3611.0,0.42,425.709991,75.783337,1011.825012,35.825001,95.339996,0.91,-0.505,28.580,-0.16,5,10,10.0
1,27.20153,-16.88955,15.0,4.6863,-3611.0,0.42,425.709991,75.783337,1011.825012,35.825001,95.339996,0.91,-0.505,28.580,-0.16,5,10,10.0
2,27.20153,-16.88955,25.0,24.4440,-3611.0,0.42,425.709991,75.783337,1011.825012,35.825001,95.339996,0.91,-0.505,28.580,-0.16,5,10,10.0
3,27.20153,-16.88955,35.0,30.9997,-3611.0,0.42,425.709991,75.783337,1011.825012,35.825001,95.339996,0.91,-0.505,28.580,-0.16,5,10,10.0
4,27.20153,-16.88955,45.0,33.6487,-3611.0,0.42,425.709991,75.783337,1011.825012,35.825001,95.339996,0.91,-0.505,28.580,-0.16,5,10,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284567,-23.19454,-40.66192,685.0,35.0402,-1894.0,0.42,425.709991,85.858334,1008.785004,35.882000,94.059998,-0.38,-0.450,28.405,-0.15,19,2,21.0
284568,-23.19454,-40.66192,695.0,45.1827,-1894.0,0.42,425.709991,85.858334,1008.785004,35.882000,94.059998,-0.38,-0.450,28.405,-0.15,19,2,21.0
284569,-23.19454,-40.66192,705.0,351.3631,-1894.0,0.42,425.709991,85.858334,1008.785004,35.882000,94.059998,-0.38,-0.450,28.405,-0.15,19,2,21.0
284570,-23.19454,-40.66192,715.0,42.6268,-1894.0,0.42,425.709991,85.858334,1008.785004,35.882000,94.059998,-0.38,-0.450,28.405,-0.15,19,2,21.0


In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

corrMatrix = final_df.corr()
plt.figure(figsize=(30, 20))
heatmap = sn.heatmap(corrMatrix, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap Sensors', fontdict={'fontsize':18}, pad=12);
plt.show()

figure = heatmap.get_figure()   
figure.savefig('corr.jpg', dpi=300, bbox_inches='tight')

In [127]:
final_df.to_csv('leg234_data.csv', index = False)